# Lab 3: Knowledge-Based Diagnostic Agent
**Course:** COME6304 - Special Topics in Software Engineering  
**Student:** Nchinde Tandjong Josue (UBa25EP063)  

## Objective
To design and implement a **Forward-Chaining Inference Engine** that simulates an automated DevOps troubleshooter. The system utilizes a Knowledge Base (KB) of production rules to diagnose server incidents based on observed symptoms.

In [1]:
# CELL 1: Knowledge Representation (Classes)
# Implements the separation of Data (Facts) and Logic (Rules)

class Fact:
    """Represents a single unit of information in the Working Memory."""
    def __init__(self, name, value):
        self.name = name
        self.value = value
    def __repr__(self): return f"({self.name}: {self.value})"
    def __eq__(self, other): return self.name == other.name and self.value == other.value
    def __hash__(self): return hash((self.name, self.value))

class Rule:
    """Represents a Production Rule: IF (conditions) THEN (result)."""
    def __init__(self, name, if_facts, then_fact, priority=1):
        self.name = name
        self.if_facts = if_facts # Antecedents
        self.then_fact = then_fact # Consequent
        self.priority = priority 

    def matches(self, current_facts):
        """Checks if all antecedents exist in the current Working Memory."""
        for req_fact in self.if_facts:
            if req_fact not in current_facts:
                return False
        return True

In [2]:
# CELL 2: The Inference Engine
# Implements Data-Driven Reasoning (Forward Chaining)

class InferenceEngine:
    def __init__(self):
        self.kb = set()       # Working Memory (Facts)
        self.rules = []       # Rule Base
        self.trace = []       # Explanation Facility (Traceability)

    def add_rule(self, rule):
        self.rules.append(rule)
        # Heuristic: Sort by priority to fire critical rules first
        self.rules.sort(key=lambda x: x.priority, reverse=True)

    def add_fact(self, name, value):
        self.kb.add(Fact(name, value))

    def clear_session(self):
        self.kb = set()
        self.trace = []

    def run(self):
        """Executes the Forward Chaining cycle until no new facts can be inferred."""
        new_fact_added = True
        while new_fact_added:
            new_fact_added = False
            for rule in self.rules:
                # Check if rule matches AND the conclusion is new knowledge
                if rule.matches(self.kb) and rule.then_fact not in self.kb:
                    self.kb.add(rule.then_fact)
                    self.trace.append(f"[LOGIC] Rule '{rule.name}' triggered -> Inferred {rule.then_fact}")
                    new_fact_added = True
                    break # Restart cycle with the new knowledge included
    
    def get_diagnosis(self):
        return [f for f in self.kb if f.name in ['diagnosis', 'action']]

In [3]:
# CELL 3: Knowledge Base Configuration
# Domain: Server Health Monitoring & Incident Response

engine = InferenceEngine()

# --- LEVEL 1: Symptom Interpretation ---
engine.add_rule(Rule("High Load Detection", 
                     [Fact("cpu", "high"), Fact("memory", "high")], 
                     Fact("state", "overloaded")))

engine.add_rule(Rule("Database Failure Detection", 
                     [Fact("latency", "high"), Fact("error", "500")], 
                     Fact("state", "db_error")))

# --- LEVEL 2: Root Cause Analysis (Chaining) ---
# These rules depend on facts inferred from Level 1

engine.add_rule(Rule("DDoS Attack Identification", 
                     [Fact("state", "overloaded"), Fact("traffic", "high")], 
                     Fact("diagnosis", "DDoS Attack")))

engine.add_rule(Rule("Memory Leak Identification", 
                     [Fact("state", "overloaded"), Fact("traffic", "normal")], 
                     Fact("diagnosis", "Memory Leak")))

engine.add_rule(Rule("Deadlock Identification", 
                     [Fact("state", "db_error"), Fact("cpu", "low")], 
                     Fact("diagnosis", "Database Deadlock")))

# --- LEVEL 3: Automated Mitigation (Action) ---
engine.add_rule(Rule("Mitigate DDoS", 
                     [Fact("diagnosis", "DDoS Attack")], 
                     Fact("action", "Enable Firewall Throttling"), priority=10))

engine.add_rule(Rule("Fix Deadlock", 
                     [Fact("diagnosis", "Database Deadlock")], 
                     Fact("action", "Restart Database Service"), priority=10))

print("✅ Knowledge Base Initialized Successfully.")

✅ Knowledge Base Initialized Successfully.


## Scenario Demonstration
The following cells demonstrate the system's ability to chain logic across multiple layers.

In [4]:
# CELL 4: Execution of Diagnostic Scenarios

def run_diagnostic_simulation(scenario_name, input_data):
    print(f"\n🔵 EXECUTING SCENARIO: {scenario_name}")
    print(f"   Input Metrics: {input_data}")
    
    # 1. Reset Memory
    engine.clear_session()
    
    # 2. Load Observed Facts
    for k, v in input_data.items():
        engine.add_fact(k, v)
        
    # 3. Run Inference
    engine.run()
    
    # 4. Display Trace (Explainability)
    print("   --- Logical Trace ---")
    if not engine.trace:
        print("   [No inferences made]")
    for step in engine.trace:
        print(f"   > {step}")
        
    # 5. Final Output
    results = engine.get_diagnosis()
    print("   --- Final Decision ---")
    if results:
        for r in results:
            print(f"   ✅ {r.name.upper()}: {r.value}")
    else:
        print("   ⚠️ Indeterminate state. Manual review required.")

# --- Define Test Vectors ---
scenario_1 = {"cpu": "high", "memory": "high", "traffic": "high"}
scenario_2 = {"latency": "high", "error": "500", "cpu": "low"}
scenario_3 = {"cpu": "high", "memory": "high", "traffic": "normal"}

# --- Run Simulations ---
run_diagnostic_simulation("Cybersecurity Threat", scenario_1)
run_diagnostic_simulation("Database Failure", scenario_2)
run_diagnostic_simulation("Application Bug", scenario_3)


🔵 EXECUTING SCENARIO: Cybersecurity Threat
   Input Metrics: {'cpu': 'high', 'memory': 'high', 'traffic': 'high'}
   --- Logical Trace ---
   > [LOGIC] Rule 'High Load Detection' triggered -> Inferred (state: overloaded)
   > [LOGIC] Rule 'DDoS Attack Identification' triggered -> Inferred (diagnosis: DDoS Attack)
   > [LOGIC] Rule 'Mitigate DDoS' triggered -> Inferred (action: Enable Firewall Throttling)
   --- Final Decision ---
   ✅ ACTION: Enable Firewall Throttling
   ✅ DIAGNOSIS: DDoS Attack

🔵 EXECUTING SCENARIO: Database Failure
   Input Metrics: {'latency': 'high', 'error': '500', 'cpu': 'low'}
   --- Logical Trace ---
   > [LOGIC] Rule 'Database Failure Detection' triggered -> Inferred (state: db_error)
   > [LOGIC] Rule 'Deadlock Identification' triggered -> Inferred (diagnosis: Database Deadlock)
   > [LOGIC] Rule 'Fix Deadlock' triggered -> Inferred (action: Restart Database Service)
   --- Final Decision ---
   ✅ DIAGNOSIS: Database Deadlock
   ✅ ACTION: Restart Database 